In [1]:
import pandas as pd

DATA = '/kaggle/input/fide-titled-chess-players/titled_players_otb.csv'
df = pd.read_csv(filepath_or_buffer=DATA)
df.head()

,fideid,name,country,sex,title,w_title,o_title,foa_title,standart_rating,rapid_rating,...,WIM,WIM_year,FM,FM_year,CM,CM_year,WGM,WGM_year,WH,WH_year
0,1701991,"Aaberg, Anton",SWE,M,IM,NaN,NaN,NaN,2322,2331,...,False,0,False,0,False,0,False,0,False,0
1,1407589,"Aabling-Thomsen, Jakob",DEN,M,IM,NaN,NaN,NaN,2327,0,...,False,0,False,2009,False,0,False,0,False,0
2,25678191,Aaditya Dhingra,IND,M,IM,NaN,NaN,NaN,2421,2208,...,False,0,False,0,False,2020,False,0,False,0
3,25778293,Aadya Gupta,IND,F,WFM,WFM,NaN,NaN,1968,1748,...,False,0,False,0,False,0,False,0,False,0
4,25991426,Aadya Ranganath,IND,F,WCM,WCM,NaN,NaN,1900,1704,...,False,0,False,0,False,0,False,0,False,0


We expect our three ratings to be pretty highly correlated. Are they?

In [2]:
df[['standart_rating', 'rapid_rating', 'blitz_rating',]].corr()

,standart_rating,rapid_rating,blitz_rating
standart_rating,1.000000,0.148530,0.154276
rapid_rating,0.148530,1.000000,0.660861
blitz_rating,0.154276,0.660861,1.000000


They are not, except for rapid/blitz. 

Let's look at the ratings distributions. We probably expect them to be exponential.

In [3]:
from plotly import express
from plotly.offline import init_notebook_mode

init_notebook_mode(connected=True)
express.histogram(data_frame=df[df['standart_rating'] > 0], x='standart_rating', facet_col='sex').show(renderer='iframe_connected',)

In [4]:
express.histogram(data_frame=df[df['rapid_rating'] > 0], x='rapid_rating', facet_col='sex').show(renderer='iframe_connected',)

In [5]:
express.histogram(data_frame=df[df['blitz_rating'] > 0], x='blitz_rating', facet_col='sex').show(renderer='iframe_connected',)

They look really Gaussian. Or at least binomial. 

Let's take our three ratings and project them into two dimensions and make a scatter plot.

In [6]:
from sklearn.manifold import TSNE

RANDOM_STATE = 2025
reducer = TSNE(random_state=RANDOM_STATE)
plot_df = pd.DataFrame(columns=['x', 'y'], data=reducer.fit_transform(X=df[['standart_rating', 'rapid_rating', 'blitz_rating',]]))
plot_df[['name', 'sex', 'standart_rating', 'rapid_rating', 'blitz_rating', ]] = df[['name', 'sex', 'standart_rating', 'rapid_rating', 'blitz_rating', ]].copy()

In [7]:
express.scatter(data_frame=plot_df, x='x', y='y', color='standart_rating', hover_name='name').show(renderer='iframe_connected',)

What do we see? We see that we have some dirty data due to zero standard ratings, but generally higher-rated players cluster together.

In [8]:
express.scatter(data_frame=plot_df, x='x', y='y', color='rapid_rating', hover_name='name').show(renderer='iframe_connected',)

This makes the initial plot easier to understand; we have a lot of players that don't play rapid chess.

In [9]:
express.scatter(data_frame=plot_df, x='x', y='y', color='blitz_rating', hover_name='name').show(renderer='iframe_connected',)

And we have a lot of players to don't play blitz. And our model picked out the cohort of players who only play standard chess over the board.

In [10]:
express.scatter(data_frame=plot_df, x='x', y='y', color='sex', hover_name='name').show(renderer='iframe_connected',)

Finally, we see that generally women are lower-rated than men, so they tend to be concentrated in the lower-rated parts of each of our distributions.